# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
# azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
# customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

In [2]:
# Read data with after memory reduction
azdias = pd.read_csv('Udacity_AZDIAS_reduce_mem_052018.csv')
customers = pd.read_csv('Udacity_CUSTOMERS_reduce_mem_052018.csv')

# Load train and test data after memory reduction
mailout_train = pd.read_csv('Udacity_MAILOUT_052018_TRAIN_reduce_mem.csv')
mailout_test = pd.read_csv('Udacity_MAILOUT_052018_TEST_reduce_mem.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Find number of rows and columns
print("Azdias shape:", azdias.shape)
print("Customers shape:", customers.shape)
print("Mailout train shape:", mailout_train.shape)
print("Mailout test shape:", mailout_test.shape)

Azdias shape: (891221, 366)
Customers shape: (191652, 369)
Mailout train shape: (42962, 367)
Mailout test shape: (42833, 366)


In [4]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [5]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [12]:
# Read information on the attributes 
# top-level list of attributes and descriptions, organized by informational category
attributes_info = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header=1)
# Detailed mapping of data values for each feature in alphabetical order
attributes_val = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header=1)

In [13]:
attributes_info.head()

,Unnamed: 0,Information level,Attribute,Description,Additional notes
0,NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,NaN,ANREDE_KZ,gender,NaN
3,NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [14]:
attributes_val.head()

,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,NaN,0,no classification possible
2,NaN,NaN,NaN,1,passive elderly
3,NaN,NaN,NaN,2,cultural elderly
4,NaN,NaN,NaN,3,experience-driven elderly


In [15]:
# Drop the unused columns
attributes_info.drop(columns=['Unnamed: 0'], inplace=True)
attributes_val.drop(columns=['Unnamed: 0'], inplace=True)

In [16]:
attributes_info.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [17]:
# Forward fill the attributes columns
attributes_val.ffill(inplace=True)

##### Fixed warnings

In [27]:
azdias.columns[18:20]

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')

In [28]:
customers.columns[18:20]

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')

In [29]:
attributes_val[attributes_val["Attribute"] == "CAMEO_DEUG_2015"]

,Attribute,Description,Value,Meaning
51,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,-1,unknown
52,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,1,upper class
53,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,2,upper middleclass
54,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,3,established middleclasse
55,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,4,consumption-oriented middleclass
56,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,5,active middleclass
57,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,6,low-consumption middleclass
58,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,7,lower middleclass
59,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,8,working class
60,CAMEO_DEUG_2015,CAMEO classification 2015 - Uppergroup,9,urban working class


In [30]:
attributes_val[attributes_val["Attribute"] == "CAMEO_INTL_2015"]

,Attribute,Description,Value,Meaning


In [31]:
attributes_val[attributes_val["Attribute"] == "CAMEO_DEUINTL_2015"]

,Attribute,Description,Value,Meaning
105,CAMEO_DEUINTL_2015,CAMEO classification 2015 - international typo...,-1,unknown
106,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,11,Wealthy Households-Pre-Family Couples & Singles
107,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,12,Wealthy Households-Young Couples With Children
108,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,13,Wealthy Households-Families With School Age Ch...
109,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,14,Wealthy Households-Older Families & Mature Co...
110,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,15,Wealthy Households-Elders In Retirement
111,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,21,Prosperous Households-Pre-Family Couples & Sin...
112,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,22,Prosperous Households-Young Couples With Children
113,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,23,Prosperous Households-Families With School Age...
114,CAMEO_DEUINTL_2015,(each German CAMEO code belongs to one interna...,24,Prosperous Households-Older Families & Mature ...


In [32]:
# Replaces suspicious data with NULL values.
azdias[['CAMEO_DEUG_2015']] = azdias[['CAMEO_DEUG_2015']].replace(['X'],np.NaN)
azdias[['CAMEO_DEUG_2015']] = azdias[['CAMEO_DEUG_2015']].replace(['XX','X'],np.NaN)
azdias['CAMEO_DEUG_2015'] = azdias['CAMEO_DEUG_2015'].astype(float)

In [33]:
azdias[['CAMEO_INTL_2015']] = azdias[['CAMEO_INTL_2015']].replace(['X'],np.NaN)
azdias[['CAMEO_INTL_2015']] = azdias[['CAMEO_INTL_2015']].replace(['XX','X'],np.NaN)
azdias['CAMEO_INTL_2015'] = azdias['CAMEO_INTL_2015'].astype(float)

In [34]:
# Replaces suspicious data with NULL values.
customers[['CAMEO_DEUG_2015']] = customers[['CAMEO_DEUG_2015']].replace(['X'],np.NaN)
customers[['CAMEO_DEUG_2015']] = customers[['CAMEO_DEUG_2015']].replace(['XX','X'],np.NaN)
customers['CAMEO_DEUG_2015'] = customers['CAMEO_DEUG_2015'].astype(float)

In [35]:
customers[['CAMEO_INTL_2015']] = customers[['CAMEO_INTL_2015']].replace(['X'],np.NaN)
customers[['CAMEO_INTL_2015']] = customers[['CAMEO_INTL_2015']].replace(['XX','X'],np.NaN)
customers['CAMEO_INTL_2015'] = customers['CAMEO_INTL_2015'].astype(float)

##### Find and drop columns we don't have any infomation about

In [36]:
features_list = attributes_val['Attribute'].unique()

In [37]:
print(len(features_list))
print(features_list)

314
['AGER_TYP' 'ALTERSKATEGORIE_GROB' 'ALTER_HH' 'ANREDE_KZ'
 'ANZ_HAUSHALTE_AKTIV' 'ANZ_HH_TITEL' 'ANZ_PERSONEN' 'ANZ_TITEL'
 'BALLRAUM' 'BIP_FLAG' 'CAMEO_DEUG_2015' 'CAMEO_DEU_2015'
 'CAMEO_DEUINTL_2015' 'CJT_GESAMTTYP' 'D19_BANKEN_ANZ_12'
 'D19_BANKEN_ANZ_24' 'D19_BANKEN_DATUM' 'D19_BANKEN_DIREKT_RZ'
 'D19_BANKEN_GROSS_RZ' 'D19_BANKEN_LOKAL_RZ' 'D19_BANKEN_OFFLINE_DATUM'
 'D19_BANKEN_ONLINE_DATUM' 'D19_BANKEN_ONLINE_QUOTE_12'
 'D19_BANKEN_REST_RZ' 'D19_BEKLEIDUNG_GEH_RZ' 'D19_BEKLEIDUNG_REST_RZ'
 'D19_BILDUNG_RZ' 'D19_BIO_OEKO_RZ' 'D19_BUCH_RZ' 'D19_DIGIT_SERV_RZ'
 'D19_DROGERIEARTIKEL_RZ' 'D19_ENERGIE_RZ' 'D19_FREIZEIT_RZ'
 'D19_GARTEN_RZ' 'D19_GESAMT_ANZ_12' 'D19_GESAMT_ANZ_24'
 'D19_GESAMT_DATUM' 'D19_GESAMT_OFFLINE_DATUM' 'D19_GESAMT_ONLINE_DATUM'
 'D19_GESAMT_ONLINE_QUOTE_12' 'D19_HANDWERK_RZ' 'D19_HAUS_DEKO_RZ'
 'D19_KINDERARTIKEL_RZ' 'D19_KONSUMTYP' 'D19_KK_KUNDENTYP'
 'D19_KOSMETIK_RZ' 'D19_LEBENSMITTEL_RZ' 'D19_LOTTO_RZ'
 'D19_NAHRUNGSERGAENZUNG_RZ' 'D19_RATGEBER_RZ' 'D19_

In [38]:
feat_no_info = list(set(azdias) - set(features_list))
print(len(feat_no_info))
print(feat_no_info)

94
['D19_VERSICHERUNGEN', 'ANZ_KINDER', 'CJT_TYP_5', 'D19_VERSI_DATUM', 'D19_TIERARTIKEL', 'KBA13_CCM_1401_2500', 'ALTER_KIND3', 'KK_KUNDENTYP', 'D19_TELKO_REST', 'D19_LOTTO', 'ALTER_KIND4', 'MOBI_RASTER', 'LNR', 'RT_SCHNAEPPCHEN', 'KONSUMZELLE', 'D19_SONSTIGE', 'VERDICHTUNGSRAUM', 'D19_KINDERARTIKEL', 'UMFELD_ALT', 'ALTER_KIND2', 'ALTER_KIND1', 'KBA13_ANTG4', 'D19_BUCH_CD', 'UMFELD_JUNG', 'VK_DISTANZ', 'GEMEINDETYP', 'HH_DELTA_FLAG', 'KBA13_ANTG1', 'D19_VERSI_ONLINE_DATUM', 'D19_WEIN_FEINKOST', 'D19_TELKO_ONLINE_QUOTE_12', 'D19_LEBENSMITTEL', 'SOHO_KZ', 'KBA13_HHZ', 'D19_BEKLEIDUNG_REST', 'D19_VERSAND_REST', 'ANZ_STATISTISCHE_HAUSHALTE', 'D19_BANKEN_GROSS', 'VK_ZG11', 'D19_SOZIALES', 'DSL_FLAG', 'KBA13_BAUMAX', 'D19_DIGIT_SERV', 'D19_KOSMETIK', 'STRUKTURTYP', 'D19_VERSI_ONLINE_QUOTE_12', 'VK_DHT4A', 'D19_VERSI_OFFLINE_DATUM', 'KBA13_ANTG2', 'CJT_KATALOGNUTZER', 'D19_BIO_OEKO', 'D19_HANDWERK', 'CJT_TYP_2', 'EINGEFUEGT_AM', 'D19_TELKO_MOBILE', 'VHA', 'ARBEIT', 'D19_BEKLEIDUNG_GEH', 'D19

In [39]:
feat_no_azdias = list(set(features_list) - set(azdias))
print(len(feat_no_azdias))
print(feat_no_azdias)

42
['D19_BANKEN_DIREKT_RZ', 'D19_LEBENSMITTEL_RZ', 'CAMEO_DEUINTL_2015', 'D19_BANKEN_LOKAL_RZ', 'D19_BANKEN_GROSS_RZ', 'HAUSHALTSSTRUKTUR', 'BIP_FLAG', 'D19_RATGEBER_RZ', 'D19_NAHRUNGSERGAENZUNG_RZ', 'D19_BUCH_RZ', 'D19_SCHUHE_RZ', 'D19_DROGERIEARTIKEL_RZ', 'D19_BANKEN_REST_RZ', 'D19_HAUS_DEKO_RZ', 'D19_VOLLSORTIMENT_RZ', 'D19_KOSMETIK_RZ', 'D19_KINDERARTIKEL_RZ', 'D19_TIERARTIKEL_RZ', 'GEOSCORE_KLS7', 'D19_SAMMELARTIKEL_RZ', 'D19_VERSICHERUNGEN_RZ', 'D19_TELKO_REST_RZ', 'WACHSTUMSGEBIET_NB', 'D19_KK_KUNDENTYP', 'D19_BEKLEIDUNG_GEH_RZ', 'D19_BILDUNG_RZ', 'D19_LOTTO_RZ', 'D19_FREIZEIT_RZ', 'D19_REISEN_RZ', 'D19_SONSTIGE_RZ', 'SOHO_FLAG', 'D19_BIO_OEKO_RZ', 'D19_ENERGIE_RZ', 'D19_GARTEN_RZ', 'D19_BEKLEIDUNG_REST_RZ', 'D19_TELKO_MOBILE_RZ', 'D19_WEIN_FEINKOST_RZ', 'D19_HANDWERK_RZ', 'D19_VERSAND_REST_RZ', 'D19_DIGIT_SERV_RZ', 'KBA13_CCM_1400_2500', 'D19_TECHNIK_RZ']


In [40]:
feat_no_info_df = pd.DataFrame(feat_no_info)
feat_no_azdias_df = pd.DataFrame(feat_no_azdias)

In [41]:
missing_info = [feat_no_info_df, feat_no_azdias_df]
missing_info_df = pd.concat(missing_info, axis=1)
missing_info_df

,0,0
0,D19_VERSICHERUNGEN,D19_BANKEN_DIREKT_RZ
1,ANZ_KINDER,D19_LEBENSMITTEL_RZ
2,CJT_TYP_5,CAMEO_DEUINTL_2015
3,D19_VERSI_DATUM,D19_BANKEN_LOKAL_RZ
4,D19_TIERARTIKEL,D19_BANKEN_GROSS_RZ
...,...,...
89,CJT_TYP_3,NaN
90,VHN,NaN
91,D19_BANKEN_REST,NaN
92,D19_ENERGIE,NaN


In [42]:
missing_info_df.to_excel('no_information.xlsx')

In [67]:
drop_col_no_info = pd.read_excel('drop_col_no_info.xlsx')
drop_col_no_info

,Unnamed: 0,no_info
0,0,AKT_DAT_KL
1,1,ALTER_KIND1
2,2,ALTER_KIND2
3,3,ALTER_KIND3
4,4,ALTER_KIND4
5,5,ALTERSKATEGORIE_FEIN
6,6,ANZ_KINDER
7,7,ANZ_STATISTISCHE_HAUSHALTE
8,8,ARBEIT
9,9,CJT_KATALOGNUTZER


In [84]:
dropping_cols = list(drop_col_no_info['no_info'].unique())
dropping_cols

['AKT_DAT_KL',
 'ALTER_KIND1',
 'ALTER_KIND2',
 'ALTER_KIND3',
 'ALTER_KIND4',
 'ALTERSKATEGORIE_FEIN',
 'ANZ_KINDER',
 'ANZ_STATISTISCHE_HAUSHALTE',
 'ARBEIT',
 'CJT_KATALOGNUTZER',
 'CJT_TYP_1',
 'CJT_TYP_2',
 'CJT_TYP_3',
 'CJT_TYP_4',
 'CJT_TYP_5',
 'CJT_TYP_6',
 'D19_KONSUMTYP_MAX',
 'D19_LETZTER_KAUF_BRANCHE',
 'D19_SOZIALES',
 'D19_TELKO_ONLINE_QUOTE_12',
 'D19_VERSI_DATUM',
 'D19_VERSI_OFFLINE_DATUM',
 'D19_VERSI_ONLINE_DATUM',
 'D19_VERSI_ONLINE_QUOTE_12',
 'DSL_FLAG',
 'EINGEFUEGT_AM',
 'EINGEZOGENAM_HH_JAHR',
 'EXTSEL992',
 'FIRMENDICHTE',
 'GEMEINDETYP',
 'HH_DELTA_FLAG',
 'KBA13_ANTG1',
 'KBA13_ANTG2',
 'KBA13_ANTG3',
 'KBA13_ANTG4',
 'KBA13_BAUMAX',
 'KBA13_GBZ',
 'KBA13_HHZ',
 'KBA13_KMH_210',
 'KOMBIALTER',
 'KONSUMZELLE',
 'LNR',
 'MOBI_RASTER',
 'RT_KEIN_ANREIZ',
 'RT_SCHNAEPPCHEN',
 'RT_UEBERGROESSE',
 'STRUKTURTYP',
 'UMFELD_ALT',
 'UMFELD_JUNG',
 'UNGLEICHENN_FLAG',
 'VERDICHTUNGSRAUM',
 'VHA',
 'VHN',
 'VK_DHT4A',
 'VK_DISTANZ',
 'VK_ZG11']

In [85]:
azdias.drop(dropping_cols, axis=1, inplace=True)

In [ ]:
# Analyse the categorical features

#function to check categorical variable counts
def cat_check_count(df, features_df):
    '''
    Takes in a features as input, if categorical then prints the counts as output
    
    '''
    categorical = features_df[features_df['type'] == 'categorical']['attribute'].values
    categorical = [x for x in categorical if x in df.columns] 
    binary = [x for x in categorical if df[x].nunique()==2]
    multilevel = [x for x in categorical if df[x].nunique()>2]
    print(df[categorical].nunique())

In [ ]:
def categorical_checker(df, attributes_df):
    '''
    Takes in a feature dataframe and a demographic dataframe and prints the counts for categorical variables
    Args:
    df: demographics dataframe
    attributes_df: dataframe with the summary of all the features
    returns:
    nothing
    '''
    categorical = attributes_df[attributes_df['type'] == 'categorical']['attribute'].values
    categorical = [x for x in categorical if x in df.columns] 
    binary = [x for x in categorical if df[x].nunique()==2]
    multilevel = [x for x in categorical if df[x].nunique()>2]
    print(df[categorical].nunique())

In [ ]:
cat_check = cat_check_count(azdias, attributes_info)

In [ ]:
missing_val = azdias.isnull().mean()
missing_val

In [ ]:
missing_val.sort_values(ascending=False).head()


In [ ]:
# Load feature info
feat_info = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header=1)

feat_info = pd.read_csv('../features.csv')
feat_info.set_index('attribute', inplace=True)

# Parse missing_or_known string into a list
def parse_missing(s):
    a = s[1:-1].split(',')
    return a

feat_info['missing_or_unknown'] = feat_info['missing_or_unknown'].apply(parse_missing)

In [ ]:
null_list = azdias.isnull().sum()(descending=True)
null_per = null_list / len(azdias) * 100
null_per[null_per > 20]

In [ ]:

# Investigate patterns in the amount of missing data in each column.
plt.figure(figsize=(10,5))
plt.title('Distribution of the Ratio of Missing Data in Each Column')
plt.hist(null_per, bins = np.linspace(10,100,19))
plt.xlabel('Percentage of Missing Data')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Distribution of the Ratio of Missing Data in Each Column (Below 20%)')
plt.hist(null_per, bins = np.linspace(0,30,15))
plt.xlabel('Percentage of Missing Data')
plt.ylabel('Count')
plt.show()

In [ ]:
len_azdias = len(azdias)
len_azdias

In [ ]:
# Drop the extra column of customers dataset.
# customers.drop(columns=['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], inplace=True)

In [ ]:
#function to check categorical variable counts
def categorical_checker(df, attributes_df):
    '''
    Takes in a feature dataframe and a demographic dataframe and prints the counts for categorical variables
    Args:
    df: demographics dataframe
    attributes_df: dataframe with the summary of all the features
    returns:
    nothing
    '''
    categorical = attributes_df[attributes_df['type'] == 'categorical']['attribute'].values
    categorical = [x for x in categorical if x in df.columns] 
    binary = [x for x in categorical if df[x].nunique()==2]
    multilevel = [x for x in categorical if df[x].nunique()>2]
    print(df[categorical].nunique())
    

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')